In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [41]:
# Carga de datos
df = pd.read_csv(r"../data/final/1_panel/final_dataset.csv")
df =df.dropna()
df.insert(0, 'status_inf', df.pop('status_inf'))
df = df.drop(columns=['conglome', 'vivienda','hogar','codperso','periodo','ocupinf_t1','ocupinf_t2','periodo'])
df

,status_inf,dpto,dominio,tuvotrabajo,sector_trabajador,categoria_trabajador,trabajopara,registropersonajuridica,cuentassunat,pagosueldo,...,tiempotrabajo,alquiler,horastotales_sempasada,ingtrabw,afp,onp,ratiodep,region,area,ciiu_6c
0,0,1,4,1,5,3,2,99.0,99.0,1,...,9.583333,400,40,13577.0,1,0,0.400000,2,1,6
1,0,1,4,1,5,3,2,99.0,99.0,1,...,1.333333,150,45,18221.0,1,0,0.800000,2,1,6
2,0,1,4,1,6,2,99,1.0,1.0,0,...,29.666666,500,52,36028.0,0,0,0.500000,2,1,4
3,0,1,4,1,4,1,99,2.0,1.0,0,...,41.750000,0,48,26510.0,0,0,0.500000,2,1,4
4,0,1,4,1,5,3,6,1.0,1.0,1,...,0.500000,100,44,12113.0,1,0,1.000000,2,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25164,1,23,3,1,6,2,99,2.0,2.0,0,...,17.000000,1000,60,24033.0,0,0,0.571429,1,1,4
25165,0,23,3,1,8,4,6,1.0,1.0,0,...,30.000000,1000,48,73477.0,1,0,0.666667,1,1,1
25166,0,23,3,1,3,3,2,99.0,99.0,1,...,2.000000,1000,42,42659.0,1,0,0.666667,1,1,6
25167,0,9,5,1,9,4,5,1.0,1.0,0,...,0.250000,18,52,19657.0,1,0,0.333333,2,0,1


In [42]:
categorical_numeric = [
    col for col in df.columns 
    if df[col].dtype in ['float64', 'int64'] and df[col].nunique() < 20
]

for col in categorical_numeric:
    df[col] = df[col].astype(int).astype(str)

In [43]:
print(df.info())
print("La distribución de la variable objetivo statu_inf es:")
print(df['status_inf'].value_counts())
print("La cantidad de preguntas de los datos originales (indep y objetivo) es",df.shape[1])

<class 'pandas.core.frame.DataFrame'>
Index: 24232 entries, 0 to 25168
Data columns (total 96 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   status_inf               24232 non-null  object 
 1   dpto                     24232 non-null  int64  
 2   dominio                  24232 non-null  object 
 3   tuvotrabajo              24232 non-null  object 
 4   sector_trabajador        24232 non-null  object 
 5   categoria_trabajador     24232 non-null  object 
 6   trabajopara              24232 non-null  object 
 7   registropersonajuridica  24232 non-null  object 
 8   cuentassunat             24232 non-null  object 
 9   pagosueldo               24232 non-null  object 
 10  pagosalario              24232 non-null  object 
 11  pagocomision             24232 non-null  object 
 12  pagodestajo              24232 non-null  object 
 13  pagosubvencion           24232 non-null  object 
 14  pagohonorarios           24

In [44]:
categoricas = ['sexo','estadocivil','tuvotrabajo','categoria_trabajador','sector_trabajador'
               ,'trabajopara','registropersonajuridica',
               'cuentassunat','tipocontrato','numpersonastrabajo','tuvootrotrabajo','normtrabaja',
               'disponiblehorastrabajar',
               'lenguamaterna','niveleduc','asiste_educ','usointernet','padece_enfer',
               'sintoma_malestar','enfermedad','recaida','accidente','notuvoenf',
               'puestosalud','centrosalud','centroopuesto','posta','hospital_minsa','hospital_seguro',
               'hospital_ffaa','consultorio','clinica','farmacia','domicilio','otro','no_busco',
               'no_dinero','lejos','demora_mucho','noconfia','nograve','remedio_caseros','no_seguro',
               'autoreceto','notiempo','maltrato','otro_motivo','essalud','seguroprivado',
               'eps','seguroffaa','sis','tipovivienda','materialparedes','materialpisos','materialtechos',
               'vivienda_status','viviendatitulo','agua_procedencia','conexionsshh','electricidad',
               'celular','internet','pobreza','dominio','dpto','ciiu_6c','area',
               'region','onp','afp',
               'pagosueldo','pagosalario','pagocomision','pagodestajo','pagosubvencion','pagohonorarios','pagoganancianegocio','pagoagropecuario','pagopropina','pagoespecie','pagootros','pagonorecibe']
# 'antepasadosconsidera','perteneceindig','registrosunat','deseaotrotrabajo','viviadistrito','agua_potable'
df_indep = df.drop(columns = ['status_inf'])
df_categoricas = df_indep[categoricas]
df_continuas = df_indep.drop(columns=categoricas)
print("Total de variables indep")
print(df_indep.shape[1])
print("La cantidad de variables indep categoricas:")
print(df_categoricas.shape[1])
print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])
for col in categoricas:
    n_unique = df_indep[col].nunique(dropna=False)
    print(f"{col}: {n_unique} valores únicos")



Total de variables indep
95
La cantidad de variables indep categoricas:
83
La cantidad de variables indep continuas:
12
sexo: 2 valores únicos
estadocivil: 6 valores únicos
tuvotrabajo: 2 valores únicos
categoria_trabajador: 5 valores únicos
sector_trabajador: 9 valores únicos
trabajopara: 6 valores únicos
registropersonajuridica: 3 valores únicos
cuentassunat: 3 valores únicos
tipocontrato: 8 valores únicos
numpersonastrabajo: 6 valores únicos
tuvootrotrabajo: 2 valores únicos
normtrabaja: 2 valores únicos
disponiblehorastrabajar: 2 valores únicos
lenguamaterna: 8 valores únicos
niveleduc: 10 valores únicos
asiste_educ: 2 valores únicos
usointernet: 2 valores únicos
padece_enfer: 2 valores únicos
sintoma_malestar: 2 valores únicos
enfermedad: 2 valores únicos
recaida: 2 valores únicos
accidente: 2 valores únicos
notuvoenf: 2 valores únicos
puestosalud: 3 valores únicos
centrosalud: 3 valores únicos
centroopuesto: 3 valores únicos
posta: 3 valores únicos
hospital_minsa: 3 valores único

In [45]:
# Crear dummies a partir de las variables categóricas
df_categoricas_encoded = pd.get_dummies(df_categoricas, columns=categoricas, drop_first=True, dummy_na=False)
print('La cantidad de variables indep categóricas después de haberse convertido en dummies es:')
print(df_categoricas_encoded.shape[1])


La cantidad de variables indep categóricas después de haberse convertido en dummies es:
230


In [46]:
# El dataset final de variables independientes (categoricas convertidas a dummies + continuas)
df_indep_final = pd.concat([df_categoricas_encoded, df_continuas], axis=1)
print('La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:')
print(df_indep_final.shape[1])


La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:
242


In [47]:
df_indep_final

,sexo_2,estadocivil_2,estadocivil_3,estadocivil_4,estadocivil_5,estadocivil_6,tuvotrabajo_2,categoria_trabajador_2,categoria_trabajador_3,categoria_trabajador_4,...,habitaciones,habitacionesdormir,combustible,personas_ingresos,personas_hogar,tiempotrabajo,alquiler,horastotales_sempasada,ingtrabw,ratiodep
0,True,True,False,False,False,False,False,False,True,False,...,5,4,2,2,5,9.583333,400,40,13577.0,0.400000
1,False,False,False,False,False,True,False,False,True,False,...,5,4,6,4,5,1.333333,150,45,18221.0,0.800000
2,False,True,False,False,False,False,False,True,False,False,...,4,2,2,2,4,29.666666,500,52,36028.0,0.500000
3,False,True,False,False,False,False,False,False,False,False,...,10,1,2,1,2,41.750000,0,48,26510.0,0.500000
4,False,False,False,False,False,True,False,False,True,False,...,6,4,2,2,2,0.500000,100,44,12113.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25164,False,True,False,False,False,False,False,True,False,False,...,6,5,2,4,7,17.000000,1000,60,24033.0,0.571429
25165,False,True,False,False,False,False,False,False,False,True,...,7,6,2,4,6,30.000000,1000,48,73477.0,0.666667
25166,False,False,False,False,False,True,False,False,True,False,...,7,6,2,4,6,2.000000,1000,42,42659.0,0.666667
25167,False,True,False,False,False,False,False,False,False,True,...,3,1,9,2,6,0.250000,18,52,19657.0,0.333333


In [48]:
df_model_logit_lasso = pd.concat([df[["status_inf"]], df_indep_final], axis=1)
df_model_logit_lasso = df_model_logit_lasso.dropna()
df_model_logit_lasso.to_csv(r'../data/final/1_panel/3_modelling/df_model_logit_lasso.csv', index=False)


In [49]:
df_model_logit_lasso


,status_inf,sexo_2,estadocivil_2,estadocivil_3,estadocivil_4,estadocivil_5,estadocivil_6,tuvotrabajo_2,categoria_trabajador_2,categoria_trabajador_3,...,habitaciones,habitacionesdormir,combustible,personas_ingresos,personas_hogar,tiempotrabajo,alquiler,horastotales_sempasada,ingtrabw,ratiodep
0,0,True,True,False,False,False,False,False,False,True,...,5,4,2,2,5,9.583333,400,40,13577.0,0.400000
1,0,False,False,False,False,False,True,False,False,True,...,5,4,6,4,5,1.333333,150,45,18221.0,0.800000
2,0,False,True,False,False,False,False,False,True,False,...,4,2,2,2,4,29.666666,500,52,36028.0,0.500000
3,0,False,True,False,False,False,False,False,False,False,...,10,1,2,1,2,41.750000,0,48,26510.0,0.500000
4,0,False,False,False,False,False,True,False,False,True,...,6,4,2,2,2,0.500000,100,44,12113.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25164,1,False,True,False,False,False,False,False,True,False,...,6,5,2,4,7,17.000000,1000,60,24033.0,0.571429
25165,0,False,True,False,False,False,False,False,False,False,...,7,6,2,4,6,30.000000,1000,48,73477.0,0.666667
25166,0,False,False,False,False,False,True,False,False,True,...,7,6,2,4,6,2.000000,1000,42,42659.0,0.666667
25167,0,False,True,False,False,False,False,False,False,False,...,3,1,9,2,6,0.250000,18,52,19657.0,0.333333


In [50]:
# Filtrar variables con un gran número de faltantes
null_percent = df_indep_final.isnull().mean()*100
cols_null = null_percent[null_percent > 80 ].index
cols_validas = null_percent[null_percent <= 80 ].index
print(cols_null)
# df_null = df[cols_validas].copy()
# df_null.shape[1]

Index([], dtype='object')


In [51]:
# Para eliminar las variables con muy poca varianza (definido por el umbral)
# Asegurar que todas las columnas dummy sean numéricas
df_indep_final = df_indep_final.apply(pd.to_numeric, errors='ignore')

varianzas = df_indep_final.var()
umbral = 0.01
vars_seleccionadas = varianzas[varianzas > umbral].index
vars_no_seleccionadas = varianzas[varianzas <= umbral].index

print("Las variables con varianza menor al umbral son:")
print(df_indep_final[vars_no_seleccionadas].nunique())
## print(df_indep_final[vars_seleccionadas].nunique())

df_indep_final=df_indep_final[vars_seleccionadas]

print("El nuevo número de variables del dataset de variables independientes es:")
print(df_indep_final.shape[1])

Las variables con varianza menor al umbral son:
categoria_trabajador_6    2
trabajopara_3             2
tipocontrato_3            2
tipocontrato_4            2
tipocontrato_5            2
lenguamaterna_3           2
lenguamaterna_5           2
lenguamaterna_6           2
lenguamaterna_7           2
lenguamaterna_8           2
centrosalud_1             2
centroopuesto_1           2
hospital_ffaa_1           2
domicilio_1               2
otro_1                    2
no_dinero_1               2
lejos_1                   2
no_seguro_1               2
maltrato_1                2
tipovivienda_5            2
tipovivienda_6            2
tipovivienda_7            2
materialparedes_2         2
materialparedes_6         2
materialparedes_8         2
materialpisos_7           2
materialtechos_6          2
materialtechos_7          2
materialtechos_8          2
vivienda_status_5         2
vivienda_status_7         2
agua_procedencia_3        2
electricidad_3            2
pagocomision_1            2


C:\Users\sergio.cuadros\AppData\Local\Temp\ipykernel_10688\3689622330.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_indep_final = df_indep_final.apply(pd.to_numeric, errors='ignore')


In [52]:
# Cálculo de la correlación entre variables y eliminar las más altas referente a un umbral (0.70):

corr_matrix = df_indep_final.corr().abs()

# 2. Seleccionamos la parte superior de la matriz de correlación
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]

print("Variables a eliminar por alta correlación (>0.60):")
print(to_drop)

df_indep_final = df_indep_final.drop(columns=to_drop)

print("Nuevo número de variables del dataset de variables independientes después de eliminas las que tienen una alta correlación:")
print(df_indep_final.shape[1])

Variables a eliminar por alta correlación (>0.60):
['trabajopara_99', 'registropersonajuridica_2', 'registropersonajuridica_99', 'cuentassunat_2', 'cuentassunat_99', 'tipocontrato_99', 'numpersonastrabajo_5', 'puestosalud_99', 'centrosalud_99', 'centroopuesto_99', 'posta_99', 'hospital_minsa_99', 'hospital_seguro_99', 'hospital_ffaa_99', 'consultorio_99', 'clinica_99', 'farmacia_99', 'domicilio_99', 'otro_99', 'no_busco_99', 'no_dinero_99', 'lejos_99', 'demora_mucho_99', 'noconfia_99', 'nograve_99', 'remedio_caseros_99', 'no_seguro_99', 'autoreceto_99', 'notiempo_99', 'maltrato_99', 'otro_motivo_99', 'essalud_2', 'eps_2', 'seguroffaa_1', 'seguroffaa_2', 'viviendatitulo_99', 'agua_procedencia_2', 'conexionsshh_2', 'pobreza_3', 'dpto_11', 'dpto_15', 'region_3', 'pagosueldo_1', 'pagosalario_1', 'pagoganancianegocio_1', 'habitacionesdormir', 'horastotales_sempasada']
Nuevo número de variables del dataset de variables independientes después de eliminas las que tienen una alta correlación:
1

In [53]:
df_model_xgb_rf = pd.concat([df[["status_inf"]], df_indep_final], axis=1)

df_model_xgb_rf.to_csv(r'../data/final/1_panel/3_modelling/df_model_xgb_rf.csv', index=False)
